In [11]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import msoffcrypto
from io import BytesIO
from openpyxl import load_workbook, Workbook

# 컬럼 매핑 규칙
COLUMN_MAPPING = {
    "수취인명": ["수취고객명", "수취인"],
    "수취인연락처1": ["수취인전화", "수취인휴대폰"],
    "우편번호": ["수취인우편번호"],
    "통합배송지": ["수취인주소"],
    "1": ["총중량"],
    "미스터불스쏘팔메토": ["상품명1"],
    "수량": ["수량1", "총수량"],
    "최종상품별총주문금액": ["물품가격1"],
    "옵션정보": ["MEMO"],
    "상품주문번호": ["고객주문번호"],
}

# 파일 경로 변수
source_file_path = None

# 파일 선택 함수
def select_file():
    global source_file_path
    source_file_path = filedialog.askopenfilename(
        title="원본 엑셀 파일 선택",
        filetypes=[("Excel files", "*.xlsx")]
    )
    if source_file_path:
        lbl_file_path.config(text=f"선택된 파일: {source_file_path}")

# 새로운 엑셀 파일 생성 함수
def generate_file():
    if not source_file_path:
        messagebox.showerror("오류", "먼저 원본 파일을 선택하세요!")
        return

    password = tk.simpledialog.askstring("비밀번호 입력", "원본 파일의 비밀번호를 입력하세요:", show='*')

    if not password:
        messagebox.showerror("오류", "비밀번호를 입력하세요!")
        return

    # 원본 파일 읽기
    try:
        decrypted = BytesIO()
        with open(source_file_path, "rb") as f:
            file = msoffcrypto.OfficeFile(f)
            file.load_key(password=password)
            file.decrypt(decrypted)

        # OpenPyXL로 해독된 파일 열기
        decrypted.seek(0)
        wb = load_workbook(decrypted)
        ws = wb.active
        data = ws.values
        columns = next(data)  # 첫 번째 행은 헤더
        source_data = pd.DataFrame(data, columns=columns)
    except Exception as e:
        messagebox.showerror("오류", f"원본 파일을 열 수 없습니다.\n에러: {e}")
        return

    # 새로운 데이터프레임 생성
    new_data = pd.DataFrame()
    for source_col, target_cols in COLUMN_MAPPING.items():
        if source_col in source_data.columns:
            for target_col in target_cols:
                new_data[target_col] = source_data[source_col]

    # 파일 저장
    save_path = filedialog.asksaveasfilename(
        title="저장할 파일 선택",
        defaultextension=".xlsx",
        filetypes=[("Excel files", "*.xlsx")]
    )
    if save_path:
        try:
            new_wb = Workbook()
            new_ws = new_wb.active
            new_ws.title = "Sheet1"

            # 컬럼명 추가
            new_ws.append(new_data.columns.tolist())

            # 데이터 추가
            for row in new_data.itertuples(index=False, name=None):
                new_ws.append(row)

            # 저장
            new_wb.save(save_path)
            messagebox.showinfo("성공", f"새로운 엑셀 파일이 저장되었습니다:\n{save_path}")
        except Exception as e:
            messagebox.showerror("오류", f"파일 저장 중 오류가 발생했습니다.\n에러: {e}")

# GUI 생성
root = tk.Tk()
root.title("엑셀 데이터 매핑 및 파일 생성")

# 파일 찾기 버튼
btn_select_file = tk.Button(root, text="파일 찾기", command=select_file)
btn_select_file.pack(pady=10)

# 선택된 파일 경로 표시
lbl_file_path = tk.Label(root, text="선택된 파일: 없음", wraplength=400)
lbl_file_path.pack(pady=10)

# 파일 생성 버튼
btn_generate_file = tk.Button(root, text="파일 생성", command=generate_file)
btn_generate_file.pack(pady=10)

# GUI 실행
root.mainloop()
